In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5),(0.5))
])

train = datasets.MNIST(
    root = "./data",
    train = True,
    download = True,
    transform = transform
)
test = datasets.MNIST(
    root = "./data",
    train = False,
    download = True,
    transform = transform
)

In [12]:
train_loader = DataLoader(train, batch_size=64, shuffle=True)
test_loader = DataLoader(test, batch_size=64, shuffle=False)

In [6]:
class CNN(nn.Module):
    
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1,8, kernel_size=3, padding = 1)
        self.pool = nn.MaxPool2d(kernel_size=2,stride=2)
        self.conv2 = nn.Conv2d(8,16,kernel_size=3, padding = 1)
        
        self.fc1 = nn.Linear(16*7*7, 64)
        self.fc2 = nn.Linear(64,10)

        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))

        x = x.view(x.size(0), -1)
        
        x = self.relu(self.fc1(x))
        x = self.fc2(x)

        return x
        

In [7]:
model = CNN().to(device)
model

CNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=784, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
  (relu): ReLU()
)

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [14]:
model.train()
epochs = 4

for epoch in range(epochs):
    total_loss=0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"EPochs_{epoch} the loss is -> {total_loss/len(train_loader):.6f}")



EPochs_0 the loss is -> 0.017740
EPochs_1 the loss is -> 0.014728
EPochs_2 the loss is -> 0.013129
EPochs_3 the loss is -> 0.011476


In [15]:
model.eval()
correct = 0
accuracy = 0


with torch.no_grad():
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, pred = torch.max(outputs, 1)

        correct += (pred == labels).sum().item()
        total += labels.size(0)

print(f'Accuracy :{100 * correct /total:.2f}%') 

Accuracy :98.77%
